In [109]:
import ssl
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import re
import statistics

In [ ]:
#To Do: get features and make data frame
#Filter for houses with greater than 3 features

In [119]:
def get_links(pages = 5):
    #gets the links from trulia properties page 
    # if length of list is zero, captcha blocked request so use a new VPN location 
    #pages: number of pages of trula to scrape
    #returns: list of links
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    i = 0
    property_links = []
    while i < pages:
        url = 'https://www.trulia.com/NY/New_York/' + "%s_p/"%str(i)
        req = Request(url, headers={"User-Agent": 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage , 'html.parser')
        for link in soup.findAll('a'):
            l = str(link.get('href'))
            if sum(map(str.isdigit, l[-17:])) == 15: property_links.append("https://www.trulia.com" + l)
        property_links = property_links[::2]
        print("finishing loop" + " " + str(i+1))
        i += 1
        time.sleep(5)
        
    return(property_links)

In [ ]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [138]:
#mean feature test
# if length of list is zero, captcha blocked request so use a new VPN location 
length = []
for url in get_links():
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()  
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    num_features = []

    for index, sou in enumerate(soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"})):
        clean_data = str(sou).split("gmLKqq\">")
        clean_data = clean_data[1].split("</span>")[0]
        num_features.append(clean_data)
    
    if len(num_features) > 0: 
        print(num_features)
        length.append(len(num_features))

print(statistics.mean(length))

finishing loop 1
finishing loop 2
finishing loop 3
finishing loop 4
finishing loop 5
['Stories: 2.5', 'Days on Market: &lt;1 Day on Trulia', 'Year Built: 1930', 'Year Updated: 1930', 'Property Type: Single Family Home', 'Price Per Sqft: $181', 'Units in Building: 1', 'Lot Area: 4320 sqft']
['Stories: 1', 'Days on Market: &lt;1 Day on Trulia', 'Year Built: 1940', 'Year Updated: 1940', 'Property Type: Single Family Home', 'Price Per Sqft: $780', 'Units in Building: 1', 'Lot Area: 4000 sqft']
['Basement: None', 'Number of Rooms: 2', 'Number of Bathrooms: 1', 'Number of Bathrooms (full): 1', 'Living Area: 489 Square Feet', 'Utilities: Public Garbage Removal', 'Appliances: Oven, Refrigerator', 'Laundry: Common Area', 'Refrigerator', 'Heating: Electric', 'Has Cooling', 'Air Conditioning: Wall Unit(s)', 'Heating Fuel: Electric', 'Flooring: Hardwood', 'Common Walls: No Common Walls', 'Stories: 1', 'Number of Stories: 14', 'Levels: One', 'Elevator', 'Floors: Hardwood', 'Exterior: Balcony', 'Has

KeyboardInterrupt: 

In [62]:
# For ignoring SSL certificate errors  (reused code)

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [63]:
#url = input("Enter URL: ")
url = 'https://www.trulia.com/p/ny/staten-island/365-victory-blvd-staten-island-ny-10301--2008726710'

In [64]:
# Use Mozilla access agent

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

In [65]:
# Creating a BeautifulSoup object of the html page for easy extraction of data.

soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
property_info = {}

In [66]:
# This code block will get you a one liner description of the listed property

for meta in soup.findAll('meta', attrs={'name': 'description'}):
    try:
        property_info['description'] = meta['content']
        break
    except: 
        pass

property_info['description']

'365 Victory Blvd, Staten Island, NY 10301 is a 2,164 sqft, 4 bed, 2 bath Foreclosure Multi-Family Home listed for $159,000. The property is Boarded so all potential buyers are asked to check with city and tax records to...'

In [67]:
#Store url 

property_info['link'] = url

property_info['link']

'https://www.trulia.com/p/ny/staten-island/365-victory-blvd-staten-island-ny-10301--2008726710'

In [68]:
#Get beds, baths, floorspace

for bed_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bedrooms"}):
    number_beds = re.findall(r"[0-9]+\s[bB]eds*", str(bed_html))
    if number_beds:
        property_info["beds"] = int(number_beds[0].split(" ")[0])

for bath_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-bathrooms"}):
    number_baths = re.findall(r"[0-9]+\s[bB]aths*", str(bath_html))
    if number_baths:
        property_info["baths"] = int(number_baths[0].split(" ")[0])

for floor_html in soup.findAll('div', attrs={'data-testid':"home-summary-size-floorspace"}):
    floor_footage = re.findall(r"[0-9]+,[0-9]+\ssqft*", str(floor_html))
    property_info["floor_space"] = {}
    if floor_footage:
        property_info["floor_space"]["value"] = int(floor_footage[0].split(" ")[0].replace(",", ""))
        property_info["floor_space"]["unit"] = floor_footage[0].split(" ")[1]

property_info


{'description': '365 Victory Blvd, Staten Island, NY 10301 is a 2,164 sqft, 4 bed, 2 bath Foreclosure Multi-Family Home listed for $159,000. The property is Boarded so all potential buyers are asked to check with city and tax records to...',
 'link': 'https://www.trulia.com/p/ny/staten-island/365-victory-blvd-staten-island-ny-10301--2008726710',
 'beds': 4,
 'baths': 2,
 'floor_space': {'value': 2164, 'unit': 'sqft'}}

In [69]:
#Extracts features for given house
for index, sou in enumerate(soup.findAll('span', attrs={"class":"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"})):
    clean_data = str(sou).split("gmLKqq\">")
    clean_data = clean_data[1].split("</span>")[0]
clean_data

'Lot Area: 3001 sqft'